In [1]:
from load_matlab_elmannet import *
import numpy as np
import gym
import time
import os
from PIL import Image

In [2]:
y_min, y_max, x_min, x_max = 25, 195, 20, 140
shape_of_single_frame = (1, (y_max-y_min),(x_max-x_min))

In [3]:
sleep_time = 0.05
model_path = "../2-elman_train/models/playM45to45F1to1000Epoch5000to10000H200/new_model.mat"

match = 45
data_path = f"../1-generate/data/match_" + str(match) + "/"

In [4]:
ACTIONS = {
        "noop": 0,
        "accelerate": 1,
        "right": 2,
        "left": 3,
        "break": 4,
        "right_break": 5,
        "left_break": 6,
        "right_accelerate": 7,
        "left_accelerate": 8,
}

In [5]:
"""ACTIONS = {
    "right": 2,
    "left": 3,
}"""

'ACTIONS = {\n    "right": 2,\n    "left": 3,\n}'

In [6]:
ACTIONS_LIST = list(ACTIONS.values())

In [7]:
def load_npz(m):
    path = data_path + "npz/"

    actions = np.load(path + 'actions.npz')
    lifes = np.load(path + 'lifes.npz')
    frames = np.load(path + 'frames.npz')
    rewards = np.load(path + 'rewards.npz')

    arr_actions = actions.f.arr_0
    arr_lifes = lifes.f.arr_0
    arr_frames = frames.f.arr_0
    arr_rewards = rewards.f.arr_0

    print("Successfully loaded NPZ.")

    return arr_actions.shape[0], arr_frames, arr_actions, arr_rewards, arr_lifes

In [8]:
num_of_frames, frames, actions, rewards, lifes = load_npz(match)

Successfully loaded NPZ.


In [9]:
frames = frames.reshape(len(frames), -1)
frames.shape

(4500, 20400)

In [10]:
def prepare_action_data(action):

    new_action = np.zeros((1, len(ACTIONS)), dtype=int) 

    new_action[0, ACTIONS_LIST.index(action)] = 1

    return new_action

In [11]:
actions

array([0, 0, 0, ..., 3, 0, 0])

In [12]:
targets = [prepare_action_data(actions[i])[0] for i in range(len(actions))]

In [13]:
targets = np.array(targets)

In [14]:
model = LoadElman(model_path)

/home/ryo/.local/lib/python3.8/site-packages/scipy/io/matlab/mio.py:226: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


U shape: (200, 20400)
W shape: (200, 200)
V shape: (9, 200)
bh shape: (200,)
bo shape: (9,)


In [15]:
model.reset_prev_s()

print_class = False

for i in range(len(frames)):
    if(print_class):
        print(i+1, np.argmax(targets[i]), "->", np.argmax(model.predict_step(frames[i]/255)))
    else:
        print(i+1, targets[i], "->", model.predict_step(frames[i]/255))

1 [1 0 0 0 0 0 0 0 0] -> [9.49388598e-01 1.34247112e-03 3.37791169e-04 3.63989192e-02
 2.56949465e-03 1.35290395e-02 4.43768195e-03 1.88188860e-03
 9.55827569e-03]
2 [1 0 0 0 0 0 0 0 0] -> [9.96103116e-01 1.45025597e-03 1.41997694e-04 4.27719990e-03
 3.86344953e-03 8.87298776e-03 3.60613075e-03 3.12749401e-03
 2.38566325e-03]
3 [1 0 0 0 0 0 0 0 0] -> [9.97029158e-01 4.86208508e-03 2.07448614e-04 2.38115424e-03
 9.85147204e-04 1.05841768e-02 2.19841287e-03 3.10488866e-03
 8.82489104e-04]
4 [1 0 0 0 0 0 0 0 0] -> [9.83512186e-01 2.44657188e-02 2.09118555e-04 1.59880142e-03
 1.86328325e-03 5.60818016e-03 1.82157217e-03 4.84034530e-03
 1.06350800e-03]
5 [1 0 0 0 0 0 0 0 0] -> [9.96732185e-01 3.77294308e-03 1.69933704e-04 1.01292449e-03
 3.10143788e-03 8.31096382e-03 4.42482699e-03 4.20486960e-03
 1.52569158e-03]
6 [1 0 0 0 0 0 0 0 0] -> [9.99402542e-01 1.36936427e-03 1.18996067e-04 5.61132247e-04
 3.08632676e-03 1.18773022e-02 5.56152954e-03 3.93500857e-03
 1.52469187e-03]
7 [1 0 0 0 0 0 0

54 [0 1 0 0 0 0 0 0 0] -> [8.28718655e-04 9.96813000e-01 1.83609681e-03 1.07736348e-03
 5.14822453e-04 1.17336639e-03 9.44843262e-04 3.40286257e-03
 8.23338996e-03]
55 [0 1 0 0 0 0 0 0 0] -> [2.24065519e-03 9.62529859e-01 3.02094753e-03 7.19276474e-04
 1.13418782e-03 2.08697210e-03 2.07440147e-03 3.06248681e-03
 7.47914797e-03]
56 [0 1 0 0 0 0 0 0 0] -> [1.04129425e-03 9.98320030e-01 1.75241029e-03 9.13027800e-04
 4.90440653e-04 1.15751132e-03 8.40962997e-04 3.40313116e-03
 5.78797180e-03]
57 [0 1 0 0 0 0 0 0 0] -> [1.47232119e-03 9.95596940e-01 4.78799171e-03 2.47900031e-04
 9.80366567e-04 1.70942226e-03 9.93067316e-04 4.58292558e-03
 8.88095920e-03]
58 [0 1 0 0 0 0 0 0 0] -> [2.76687601e-04 9.99264084e-01 4.60592621e-03 2.68113463e-04
 6.12376850e-04 1.80823727e-03 1.52770102e-03 7.43586473e-03
 8.04293015e-03]
59 [0 1 0 0 0 0 0 0 0] -> [1.39778446e-04 9.99422157e-01 2.98371954e-02 3.46943745e-04
 2.57832629e-04 1.64543875e-03 1.33350818e-03 5.55112613e-03
 1.03932685e-02]
60 [0 1 0 

104 [0 1 0 0 0 0 0 0 0] -> [7.29356957e-03 9.97874470e-01 7.03631971e-03 3.84991482e-04
 8.13522599e-04 2.74086956e-03 1.09737885e-03 5.34780063e-03
 1.18664052e-02]
105 [0 1 0 0 0 0 0 0 0] -> [1.25356648e-02 9.93644480e-01 3.85557146e-03 1.49890153e-04
 1.07606860e-03 3.39853216e-03 9.19776203e-04 7.10913815e-03
 4.40883266e-03]
106 [0 1 0 0 0 0 0 0 0] -> [1.53997099e-02 9.95373206e-01 2.78908001e-03 7.01354893e-04
 2.60816157e-03 2.60456966e-03 8.57022427e-04 9.82844251e-03
 3.39701667e-03]
107 [0 1 0 0 0 0 0 0 0] -> [0.00221076 0.99862513 0.00171314 0.00119481 0.00238861 0.00284289
 0.00110008 0.00510015 0.0066607 ]
108 [0 1 0 0 0 0 0 0 0] -> [1.86715719e-03 9.98507604e-01 3.33452110e-03 1.19108217e-03
 1.12956638e-03 2.84527000e-03 8.13647323e-04 4.13487009e-03
 9.12180223e-03]
109 [0 1 0 0 0 0 0 0 0] -> [1.71486910e-03 9.99077885e-01 1.24878061e-03 1.18076019e-02
 5.66818151e-04 1.21309609e-03 5.25247277e-04 3.74993852e-03
 3.68770894e-03]
110 [0 1 0 0 0 0 0 0 0] -> [2.36508129e-0

159 [1 0 0 0 0 0 0 0 0] -> [0.99436091 0.01058654 0.0019576  0.0010394  0.00329904 0.00706526
 0.00170388 0.00416241 0.00326408]
160 [1 0 0 0 0 0 0 0 0] -> [9.83941404e-01 1.71732262e-02 3.75980968e-03 4.89267442e-04
 4.65930168e-03 5.79118078e-03 2.72718212e-03 1.93640998e-03
 2.90001602e-03]
161 [1 0 0 0 0 0 0 0 0] -> [9.84222424e-01 1.75095786e-02 9.16660425e-03 3.62537999e-04
 4.51696097e-03 5.71015588e-03 1.55217195e-03 1.70861125e-03
 2.71314247e-03]
162 [1 0 0 0 0 0 0 0 0] -> [0.98201738 0.00406346 0.00703869 0.00217307 0.00653732 0.00874712
 0.00212083 0.00384921 0.00481109]
163 [1 0 0 0 0 0 0 0 0] -> [9.95049952e-01 3.07600112e-03 3.59181364e-03 5.96032756e-04
 2.05918857e-03 6.51300093e-03 7.92256734e-04 1.74408206e-03
 2.29897323e-03]
164 [1 0 0 0 0 0 0 0 0] -> [9.94665594e-01 5.94097143e-03 2.59125151e-03 3.17694111e-04
 2.50181428e-03 4.40258913e-03 2.54669928e-04 5.37566474e-03
 3.62278138e-03]
165 [1 0 0 0 0 0 0 0 0] -> [9.75764475e-01 6.99691103e-02 4.92653909e-03 2.479

220 [1 0 0 0 0 0 0 0 0] -> [9.51572303e-01 6.13163267e-03 5.26831503e-02 1.63500815e-04
 6.99906423e-04 1.37827559e-03 6.44432867e-04 4.99214080e-03
 3.08916100e-03]
221 [1 0 0 0 0 0 0 0 0] -> [9.16876433e-01 4.73078130e-03 1.06480934e-01 1.36958690e-04
 1.06333028e-03 1.48277943e-03 5.46127414e-04 1.12951642e-02
 4.28267336e-03]
222 [0 0 1 0 0 0 0 0 0] -> [1.02994697e-01 3.68012222e-02 8.03871889e-01 4.47532154e-04
 2.71679621e-04 1.53917962e-03 5.13614196e-04 7.09893702e-03
 2.92372512e-03]
223 [0 0 1 0 0 0 0 0 0] -> [2.93693942e-02 3.91251420e-02 9.15773557e-01 5.14359752e-03
 4.02233564e-04 3.76414978e-04 1.83026733e-04 3.10681126e-03
 8.69608003e-04]
224 [0 0 1 0 0 0 0 0 0] -> [5.64061427e-02 4.64679738e-03 8.52445230e-01 6.30903866e-03
 7.41874173e-04 1.47445692e-03 1.99220362e-03 3.56588379e-03
 5.59939911e-04]
225 [1 0 0 0 0 0 0 0 0] -> [9.59280198e-01 1.92147271e-04 6.40522314e-02 6.06671814e-03
 1.06669158e-03 4.55512883e-03 1.44862001e-03 2.39270613e-03
 5.43549672e-04]
226 

...

2347 [0 1 0 0 0 0 0 0 0] -> [3.25843418e-03 9.88930498e-01 3.75024268e-01 7.91650622e-04
 1.24085838e-03 1.17031389e-04 2.88586470e-02 1.36526778e-02
 5.76312914e-02]
2348 [0 1 0 0 0 0 0 0 0] -> [1.66842601e-02 9.71861435e-01 1.75217467e-01 5.04178393e-04
 1.97938456e-03 8.51369785e-05 1.25911838e-02 2.84712225e-02
 9.66566485e-02]
2349 [0 1 0 0 0 0 0 0 0] -> [1.28652811e-02 9.80001693e-01 6.40645792e-02 1.14727595e-03
 9.74952617e-04 1.27050415e-04 1.24918671e-02 1.97970185e-02
 2.22001542e-02]
2350 [1 0 0 0 0 0 0 0 0] -> [3.04908397e-02 9.88728704e-01 2.28283696e-02 2.50866575e-03
 4.12070921e-03 9.86927743e-05 3.25251680e-02 2.70713708e-02
 1.34886600e-02]
2351 [0 0 1 0 0 0 0 0 0] -> [3.05085306e-02 9.78528352e-01 3.75970567e-02 9.37956292e-04
 2.90145592e-03 2.79618854e-04 3.30358179e-02 2.92404049e-02
 1.92833550e-02]
2352 [0 0 1 0 0 0 0 0 0] -> [8.33212146e-03 9.31592936e-01 2.00285486e-01 1.55880376e-03
 7.14446980e-04 1.02120404e-04 3.53832002e-02 2.06578813e-02
 1.97973921e-02

2410 [0 1 0 0 0 0 0 0 0] -> [9.93923256e-05 1.77127646e-01 4.68941588e-01 1.36071534e-02
 2.32009449e-03 2.89904905e-04 1.04405279e-02 6.71635228e-02
 9.50278881e-01]
2411 [0 1 0 0 0 0 0 0 0] -> [2.43964186e-04 1.46890601e-01 4.27377332e-01 6.12133889e-03
 3.61910694e-03 6.75272207e-04 6.39401465e-03 1.10455447e-01
 9.06051540e-01]
2412 [0 1 0 0 0 0 0 0 0] -> [7.78438143e-04 9.64412574e-02 3.02227605e-01 6.78669708e-03
 2.88786587e-03 2.56007916e-04 1.80620632e-02 2.01291644e-01
 9.68589914e-01]
2413 [0 1 0 0 0 0 0 0 0] -> [6.30292026e-04 1.72062520e-01 3.45691422e-01 4.79778451e-03
 4.53687913e-03 2.60640800e-04 1.06714966e-02 1.83725585e-01
 9.74938511e-01]
2414 [0 1 0 0 0 0 0 0 0] -> [3.61715370e-03 1.50423021e-01 3.77457208e-01 1.88569076e-03
 4.67847664e-03 3.48453951e-04 1.79603093e-02 1.51257199e-01
 9.72902026e-01]
2415 [0 1 0 0 0 0 0 0 0] -> [0.00110957 0.2929742  0.18229889 0.02692912 0.00582849 0.00098921
 0.01360896 0.17552531 0.91696811]
2416 [0 1 0 0 0 0 0 0 0] -> [1.1744

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



2461 [1 0 0 0 0 0 0 0 0] -> [0.0240382  0.00436135 0.17814496 0.10049548 0.00132217 0.00179168
 0.01715644 0.02983805 0.6524787 ]
2462 [1 0 0 0 0 0 0 0 0] -> [0.02050251 0.0035578  0.47679451 0.07506552 0.0010126  0.00094343
 0.0168331  0.01961089 0.7327484 ]
2463 [1 0 0 0 0 0 0 0 0] -> [0.00606104 0.00848414 0.42499322 0.10122125 0.00180267 0.00077888
 0.03666048 0.02609685 0.58873655]
2464 [1 0 0 0 0 0 0 0 0] -> [0.0077318  0.00715524 0.57248539 0.04507054 0.00130387 0.00138328
 0.03663836 0.0220605  0.61929477]
Traceback (most recent call last):
  File "/home/ryo/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-15-7dfda80de69d>", line 9, in <module>
    print(i+1, targets[i], "->", model.predict_step(frames[i]/255))
  File "/mnt/c/Users/alece/Desktop/UnB/tcc/rnn_enduro/3-load_model/load_matlab_elmannet.py", line 34, in predict_step
    mulw = np.dot(self.W, self.p

TypeError: object of type 'NoneType' has no len()

In [ ]:
frames[0]/255

In [ ]:
model.reset_prev_s()
predicted = []
correct = 0
wrong = 0
for i in range( len(frames) ):
    
    if np.argmax(targets[i]) == np.argmax(model.predict_step(frames[i]/255)):
        correct += 1
    else:
        wrong +=1

In [ ]:
print(correct)
print(wrong)

In [ ]:
#798/(798+202)

In [16]:
env = gym.make("Enduro-v0")

In [17]:
frame = env.reset()

In [18]:
reward, action, done, info = 0, 0, False, {'ale.lives': 0}

In [19]:
frame.shape

(210, 160, 3)

In [20]:
model.reset_prev_s()

In [21]:
while(True):
    time.sleep(sleep_time)
    env.render()
    
    frame = frame[y_min:y_max, x_min:x_max]

    frame = Image.fromarray(frame)
    frame = frame.convert("L")
    
    frame = np.asarray(frame)
    frame = frame.reshape(20400,)
    frame = frame/255
    
    action = model.predict_step(frame)
    action = list(ACTIONS.values())[np.argmax(action, axis=0)]
    
    print(action)
    
    frame, reward, done, info = env.step(action)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
0
0
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
8
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt

